In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
from pyMongoCrud import AnimalShelter

# Login credentials
username = "ZEUS"
password = "75204775ImBack"
db = AnimalShelter(username, password)

# Initial data retrieval
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(html.B(html.H1('By: Christopher Williams?')), style={'text-align': 'right'}),
    html.Hr(),
    #The buttons
    html.Div(className='buttonRow', style={'display': 'flex'}, children=[
        html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
        html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
        html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
        html.Button(id='submit-button-four', n_clicks=0, children='Reset')
    ]),
    #The main data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],         #First row selected by default
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    #Create the bar chart
    html.Div(id='chart-id',className='col s12 m6',
    children=[dcc.Graph(id='bar-chart')]),
    #Create the geolocation chart
    html.Div(id='map-id', className='col s12 m6', children=[
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
               ])
    ]),
    #Display Salvare Logo
    html.Img(src='Grazioso Salvare Logo.png', style={'width': '100%', 'height': 'auto'})
])


# Interaction Between Components / Controller
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')]
)
def update_data_table(button1_clicks, button2_clicks, button3_clicks, button4_clicks):
    # Determine which button was clicked based on the highest n_clicks value
    clicked_button = max(
        ("button1", button1_clicks),
        ("button2", button2_clicks),
        ("button3", button3_clicks),
        ("button4", button4_clicks),
        key=lambda x: x[1]
    )[0]

    if clicked_button == "button1":  # Water Rescue
        df_filtered = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))

    elif clicked_button == "button2":  # Mountain Rescue
        df_filtered = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", " Old English Sheepdog",
                              " Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))

    elif clicked_button == "button3":  # Disaster Rescue
        df_filtered = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }))
    else:  # Reset or initial state
        df_filtered = pd.DataFrame.from_records(db.read({}))

    df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')


# Separate callback for the bar chart
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('datatable-id', 'data')]
)

def update_bar_chart(data):
    if not data:
        # Return an empty figure if there's no data
        return px.bar()

    df = pd.DataFrame.from_records(data)
    bar_chart = px.bar(df, x='breed', title='Animal Type Distribution')
    return bar_chart


@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'selected_rows')]
)
def update_map(selected_row):
    if selected_row is None or not selected_row:
        return []

    selected_data = df.iloc[selected_row[0]]

    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=[selected_data[13], selected_data[14]], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(
                       position=[selected_data[13], selected_data[14]],
                       children=[
                           dl.Tooltip(selected_data[4]),
                           dl.Popup([
                               html.H1("Animal Name"),
                               html.P(selected_data[9])
                           ])
                       ]
                   )
               ])
    ]


# Run the app in another window
app.run_server(debug=True)


Dash app running on http://127.0.0.1:16472/
